In [ ]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

# Set up the scope for the APIs
scope = ["https://www.googleapis.com/auth/spreadsheets.readonly", 
         "https://www.googleapis.com/auth/drive.readonly"]

# Provide the path to the credentials JSON file you downloaded
creds = ServiceAccountCredentials.from_json_keyfile_name('/Users/alexandercappelen/Documents/keys/frb-elite-88e4dcc7ec5c.json', scope)

# Authorize and create the client
client = gspread.authorize(creds)

# Open the sheet by name
player_data = client.open("frb-volley-game-stats").worksheet("player-offense")

# Fetch all records (rows) from the sheet
player_data = player_data.get_all_records()

# Print the data or convert it into a DataFrame
import pandas as pd
df = pd.DataFrame(player_data)
df

,date,match-id,match,set,player,serves,aces,serve-error,points-won,point-scoring-pct,attack,kills,attack-errors,kill-pct,kill-effic
0,08/12/2024,141108,frb-gentofte,1,Lasse,2,1,0,1,50.0,1,1,0,100.000000,1.000000
1,08/12/2024,141108,frb-gentofte,1,Nico,0,0,0,0,0.0,0,0,0,0.000000,0.000000
2,08/12/2024,141108,frb-gentofte,1,Kristian,5,0,2,3,60.0,2,1,0,50.000000,0.500000
3,08/12/2024,141108,frb-gentofte,1,Ando,0,0,0,0,0.0,0,0,0,0.000000,0.000000
4,08/12/2024,141108,frb-gentofte,1,Bosse,0,0,0,0,0.0,0,0,0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,06/10/2024,141067,frb-kv61,3,Mads,5,0,1,3,60.0,0,0,0,0.000000,0.000000
376,06/10/2024,141067,frb-kv61,3,Martin,0,0,0,0,0.0,7,3,2,42.857143,0.142857
377,06/10/2024,141067,frb-kv61,3,Mikkel,0,0,0,0,0.0,0,0,0,0.000000,0.000000
378,06/10/2024,141067,frb-kv61,3,Frederik,0,0,0,0,0.0,0,0,0,0.000000,0.000000


In [16]:
df["error-pct"] = df["attack-errors"] / df["attack"]

summary = df.groupby(['match', 'player']).agg(
    attack_attempts=('attack', 'sum'),
    total_kills=('kills', 'sum'),
    attack_errors=('attack-errors', 'sum')
).reset_index()

summary["kill_pct"] = ((summary["total_kills"]) / summary["attack_attempts"] * 100).round(0)
summary["error_pct"] = ((summary["attack_errors"]) / summary["attack_attempts"] * 100).round(0)
summary["kill_effic"] = ((summary["total_kills"] - summary["attack_errors"]) / summary["attack_attempts"])#.round(3)
summary = summary.dropna()
summary["kill_pct"] = summary["kill_pct"].astype(int)
summary["error_pct"] = summary["error_pct"].astype(int)
summary["kill_effic"] = summary["kill_effic"].round(3)

import json

# Convert DataFrame to JSON
data = summary.to_dict(orient="records")  # Convert DataFrame rows to list of dictionaries
with open("../player-offense-per-game.json", "w") as f:
    json.dump(data, f, indent=4)